# Musformer

In [1]:
!pip install kagglehub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.4/63.4 kB 2.8 MB/s eta 0:00:00


In [3]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
jakerr5280_musdb18_music_source_separation_dataset_path = kagglehub.dataset_download('jakerr5280/musdb18-music-source-separation-dataset')

print('Data source import complete.')


import os
db_path = '/notebooks/data'
output_path = '/notebooks/results'


# Create destination if needed
os.makedirs(db_path, exist_ok=True)


# Create destination if needed
os.makedirs(output_path, exist_ok=True)

100%|██████████| 4.06G/4.06G [00:51<00:00, 84.2MB/s]

Extracting files...


Data source import complete.


In [16]:
import csv
from pathlib import Path

def create_metrics_csv(file_path):
    headers = [
        "model_name", "n_epochs", "learning_rate", "chunk_size", "sample_rate", "sdr", "sdr_i", "si-snr", "si-snr_i"
    ]
    
    Path(file_path).parent.mkdir(parents=True, exist_ok=True)

    with open(file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(headers)
    
    print(f"CSV file created successfully at: {file_path}")

if result_file_path is not None:
    create_metrics_csv(result_file_path)

CSV file created successfully at: /notebooks/metrics.csv


In [1]:
db_path = '/notebooks/data'
output_path = '/notebooks/results'
result_file_path = '/notebooks/metrics.csv'



In [4]:
import os
import shutil

try:
    shutil.move(jakerr5280_musdb18_music_source_separation_dataset_path + '/train', db_path)
    shutil.move(jakerr5280_musdb18_music_source_separation_dataset_path + '/test', db_path)  # Moves the entire directory
    print(f"Moved {jakerr5280_musdb18_music_source_separation_dataset_path} to {db_path}")
except Exception as e:
    print(f"Error: {e}")


Moved /root/.cache/kagglehub/datasets/jakerr5280/musdb18-music-source-separation-dataset/versions/1 to /notebooks/data


In [8]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk(db_path):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/notebooks/data/train/James May - Dont Let Go.stem.mp4
/notebooks/data/train/North To Alaska - All The Same.stem.mp4
/notebooks/data/train/Titanium - Haunted Age.stem.mp4
/notebooks/data/train/Music Delta - Gospel.stem.mp4
/notebooks/data/train/Drumtracks - Ghost Bitch.stem.mp4
/notebooks/data/train/A Classic Education - NightOwl.stem.mp4
/notebooks/data/train/Leaf - Wicked.stem.mp4
/notebooks/data/train/Dreamers Of The Ghetto - Heavy Love.stem.mp4
/notebooks/data/train/Remember December - C U Next Time.stem.mp4
/notebooks/data/train/Port St Willow - Stay Even.stem.mp4
/notebooks/data/train/Patrick Talbot - A Reason To Leave.stem.mp4
/notebooks/data/train/Meaxic - You Listen.stem.mp4
/notebooks/data/train/Actions - South Of The Water.stem.mp4
/notebooks/data/train/Helado Negro - Mitad Del Mundo.stem.mp4
/notebooks/data/train/ANiMAL - Clinic A.stem.mp4
/notebooks/data/train/James May - All Souls Moon.stem.mp4
/notebooks/data/train/James May - On The Line.stem.mp4
/notebooks/data/train/J

## Musformer
Although misleading, but the name seemed nice. The goal of this project is to try out multiple source separation models in speechbrain. The idea is to replicate building a model in the waveform domain directly. We see that multiple models try to solve the music sourse separation as a problem and most state-of-the-art models reach an SiSDR ratio of 9 to tackle the same. 

Most models working on this problem came into existence as a solution to the SDX (Sound Demixing) challenge. A few of the existing solutions are as follows: 
- **MMDenseLSTM**: Combines dense blocks with LSTMs for lightweight waveform separation.
- **Demucs (v1/v2)**: U-Net with bidirectional LSTMs; later versions add transformers.
- **ConvTasNet**: Efficient temporal convolutional network with learned encoder/decoder.
- **DPRNNet**: Dual-path RNN with attention, offering SOTA results at higher compute costs.
- **BandSplitRNN**: Separates audio into frequency bands processed by independent RNNs before recombination.
- **Wave-U-Net**: Adapts medical imaging's U-Net architecture for waveform-based source separation with learned down/upsampling.
- **Open-Unmix**: Spectrogram-based separation model using three-layer BiLSTMs with industry-standard implementation.
- **ResUNetDecouple**: U-Net variant with residual connections that decouples magnitude and phase processing.
- **TDCN++**: Improved temporal convolutional network with global skip connections and stacked dilation patterns.
- **Spleeter**: Facebook's lightweight CNN-based separator using spectrogram masking with pretrained models.

Of these, two models which catch one's eye are convtasnet, which seemed to provide good results with the most optimally efficient architecture and demucs, which uses an advanced convolutional architecure built on top of Wave-U-Net and stretches its performance to state-of-the-art levels.

## Disclaimers
This notebook refers to the tip of the iceberg of multiple approaches tried along the way in order to reach here. It provides for a summary of the underlying work done around trials and errors in speech separation. More about the same is appended in the Archives section at the end of this report notebook 

## Step1 : Gaining Context (Literature Review)

### A bit about the Dataset
First things first, lets talk about the dataset and the models a bit. MUSDB18 is the benchmark dataset for music source separation, containing 150 full-track recordings (100 for training, 50 for test) with isolated stems for vocals, drums, bass, and other instruments. It provides professionally mixed 44.1kHz stereo audio, enabling evaluation of waveform-domain separation models. The dataset covers diverse genres and production styles, making it ideal for testing real-world generalization. It has become the standard benchmark for models like Open-Unmix, Demucs, and D3Net, with SI-SDR and SDR as primary metrics. The included Python toolbox (musdb) provides data loading, evaluation, and stem mixing utilities. Musdb is however dependent on something known as STEM files i.e. music tensor files combined and compressed into one. We use the kaggle uploaded version of the dataset which saves us the trouble of downloading, unzipping and uploading the same. You can find the dataset here: https://www.kaggle.com/datasets/jakerr5280/musdb18-music-source-separation-dataset

### A bit about the models


#### Demucs v2
- Key Idea: A hybrid U-Net + Bi-LSTM architecture that processes raw waveforms.
- U-Net Structure: Encoder-decoder with skip connections for multiscale feature extraction.
- Bidirectional LSTMs: Added between encoder and decoder to capture long-term temporal relationships.

Improvements in v2:
- Deeper architecture with more layers.
- Better training strategies (e.g., dynamic mixing).
- Optional transformer layers in later variants.

Advantages:
- Strong separation quality, especially for music.
- Handles variable-length inputs well.

Limitations:
- Computationally heavier than ConvTasNet due to LSTMs.


## Step 2: Installing Stuff and Basic Setup


In [2]:
%%capture
!pip install musdb
!pip install mir_eval
!pip install museval

# Installing SpeechBrain via pip
BRANCH = 'develop'
!python -m pip install git+https://github.com/speechbrain/speechbrain.git@$BRANCH

# Clone SpeechBrain repository
!git clone https://github.com/speechbrain/speechbrain/

In [2]:
import os
import numpy as np
np.float_ = np.float64
import musdb

MUS_DB_PATH = db_path

mus = musdb.DB(root=MUS_DB_PATH)
mus_train = musdb.DB(root=MUS_DB_PATH,subsets="train", split="train")
mus_valid = musdb.DB(root=MUS_DB_PATH,subsets="train", split="valid")
mus_test = musdb.DB(root=MUS_DB_PATH,subsets="test")
print(mus_train[0])
print(mus_test[0])

NameError: name 'db_path' is not defined

In [3]:
db_path = '/notebooks/data'
output_path = '/notebooks/results'
result_file_path = '/notebooks/metrics.csv'



In [4]:
%%file dataset.py


import csv
import os
import sys

import torch
import torch.nn.functional as F
import torchaudio
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import musdb
import numpy as np
import speechbrain as sb
import psutil


class MusDBDataset(Dataset):
    def __init__(self, root, subset="train", split=None, target_sr=8000, chunk_size=15):
        """
        True lazy-loading for MUSDB
        :param chunk_size: in seconds
        """
        if split is not None:
            self.db = musdb.DB(root=root, subsets=subset, split=split, is_wav=False)
        else:
            self.db = musdb.DB(root=root, subsets=subset, is_wav=False)
        self.target_sr = target_sr
        self.chunk_size = chunk_size
        self.tracks = [{
            "path": track.path,
            "duration": track.duration,
            "rate": track.rate,
            "stem_id": f"{track.name}_{track.stem_id}" # Needed for STEM access
        } for track in self.db.tracks]

    def __len__(self):
        return len(self.tracks)

    def __getitem__(self, idx):
        track_info = self.tracks[idx]
        
        # Load chunk directly from disk without full track loading
        def load_stem_chunk(stem_name, random_chunk_val=0):
            # MUSDB's internal lazy loading
            track = self.db.tracks[idx]
            if stem_name == "mix":
                source = track
            else:
                source = track.targets[stem_name]
            
            # Calculate chunk bounds
            start = random_chunk_val
            stop = start + self.chunk_size * track_info["rate"]
            
            # Load only the needed segment
            audio = source.audio[start:stop]
            
            # Convert and resample
            audio_tensor = torch.from_numpy(audio).float().permute(1, 0)
            return torchaudio.functional.resample(
                audio_tensor,
                orig_freq=track_info["rate"],
                new_freq=self.target_sr
            ).mean(dim=0, keepdim=False)
            
        
            # chunk_size = orig_sr * chunk_size_seconds  # Convert chunk size to samples
        
            # resampled_chunks = []
        
            # for i in range(0, audio_tensor.shape[1], chunk_size):
            #     chunk = audio_tensor[:, i:i + chunk_size]  # Extract chunk
            #     resampled_chunk = torchaudio.functional.resample(chunk, orig_freq=orig_sr, new_freq=target_sr)
            #     resampled_chunks.append(resampled_chunk)
        
            # # Concatenate back the processed chunks
            # # print("PROCESSING CHUNKS")
            # resampled_audio = torch.cat(resampled_chunks, dim=1)
            # # print(resampled_audio.shape)
            # resampled_audio = resampled_audio.mean(dim=0, keepdim=False)
            # # print(resampled_audio.shape)
            # # print(resampled_audio.shape)
            # return resampled_audio
        # if random_chunk_start_val:
        
        random_chunk_start_val = 0
        
        return {
            "mix_sig": load_stem_chunk("mix", random_chunk_start_val),
            "voc_sig": load_stem_chunk("vocals",random_chunk_start_val),
            "inst_sig": load_stem_chunk("accompaniment",random_chunk_start_val),
            "track_id": track_info['stem_id']
        }

Overwriting dataset.py


## Step 3: Building ConvTasNet

#### ConvTasNet
- Key Idea: A fully convolutional, end-to-end waveform model that avoids spectrograms entirely.
- Encoder-Decoder: Uses 1D convolutions to learn a latent representation of the waveform.
- Temporal Convolutional Network (TCN): Processes the latent features with stacked dilated convolutions for long-range dependencies.
- Mask Estimation: Applies a soft mask in the latent space to separate sources.

Advantages:
- Lightweight and parallelizable (no RNNs).
- Strong performance on speech and music separation.

Limitations:
- May struggle with very long-term dependencies due to fixed receptive fields.

Personal Interests:
Convtasnet proves to be one of the smallest models which generally speaking outperforms most other state-of-the-art models (in the waveform domain). This is intruiging because it uses a simple encoder decoder architecture with a masknet type bottleneck. The version we use in this has 3.3 Million parameters and is successfully able to detach vocals from its accompaniements

Reference and Sources:


In [32]:
%%file convtasnet-hparams.yaml
# ################################
# Model: ConvTasNet for Music Vocal Separation
# https://arxiv.org/abs/2010.13154
# Dataset : Musdb
# ################################
# Basic parameters
# Seed needs to be set at top of yaml, before objects with parameters are made

seed: 1234
__set_seed: !apply:speechbrain.utils.seed_everything [!ref <seed>]

# Data params
data_folder: !PLACEHOLDER


experiment_name: convtasnet
output_folder: !ref /notebooks/results/<experiment_name>/<seed>
train_log: !ref <output_folder>/train_log.txt
save_folder: !ref <output_folder>/save
train_data: !ref <output_folder>/train.json
valid_data: !ref <output_folder>/valid.json
test_data: !ref <output_folder>/test.json
skip_prep: False
db_path: '/notebooks/data'

result_file_path: !PLACEHOLDER


# Experiment params
precision: fp16 # bf16, fp16 or fp32

instrumental_classification: False
noprogressbar: False
save_audio: True # Save estimated sources on disk
sample_rate: 8000
n_audio_to_save: 10
chunk_size: 20

####################### Training Parameters ####################################
N_epochs: 30
batch_size: 1
lr: 0.00015
clip_grad_norm: 5
loss_upper_lim: 999999  # this is the upper limit for an acceptable loss
num_sources: 2


# loss thresholding -- this thresholds the training loss
threshold_byloss: True
threshold: -30

# Encoder parameters
N_encoder_out: 256
# out_channels: 256
kernel_size: 16
kernel_stride: 8

# Dataloader options
dataloader_opts:
    batch_size: !ref <batch_size>
    num_workers: 1


# Specifying the network
Encoder: !new:speechbrain.lobes.models.dual_path.Encoder
    kernel_size: !ref <kernel_size>
    out_channels: !ref <N_encoder_out>


MaskNet: !new:speechbrain.lobes.models.conv_tasnet.MaskNet
    N: 256
    B: 256
    H: 512
    P: 3
    X: 6
    R: 4
    C: !ref <num_sources>
    norm_type: 'gLN'
    causal: True
    mask_nonlinear: 'relu'


Decoder: !new:speechbrain.lobes.models.dual_path.Decoder
    in_channels: !ref <N_encoder_out>
    out_channels: 1
    kernel_size: !ref <kernel_size>
    stride: !ref <kernel_stride>
    bias: False

optimizer: !name:torch.optim.Adam
    lr: !ref <lr>
    weight_decay: 0

loss: !name:speechbrain.nnet.losses.get_si_snr_with_pitwrapper

lr_scheduler: !new:speechbrain.nnet.schedulers.ReduceLROnPlateau
    factor: 0.5
    patience: 2
    dont_halve_until_epoch: 85

epoch_counter: !new:speechbrain.utils.epoch_loop.EpochCounter
    limit: !ref <N_epochs>

modules:
    encoder: !ref <Encoder>
    decoder: !ref <Decoder>
    masknet: !ref <MaskNet>

checkpointer: !new:speechbrain.utils.checkpoints.Checkpointer
    checkpoints_dir: !ref <save_folder>
    recoverables:
        encoder: !ref <Encoder>
        decoder: !ref <Decoder>
        masknet: !ref <MaskNet>
        counter: !ref <epoch_counter>
        lr_scheduler: !ref <lr_scheduler>

train_logger: !new:speechbrain.utils.train_logger.FileTrainLogger
    save_file: !ref <train_log>

Overwriting convtasnet-hparams.yaml


In [35]:
%%file convtasnet-train.py
#!/usr/bin/env/python3
"""
Recipe for vocal separation using convtasnet
"""

import csv
import os
import sys

import numpy as np
import torch
import torch.nn.functional as F
import torchaudio
from hyperpyyaml import load_hyperpyyaml
from tqdm import tqdm
import pdb

import musdb
import torchaudio
import numpy as np
from torch.utils.data import Dataset
import speechbrain as sb
import psutil
from dataset import MusDBDataset


import speechbrain as sb
import speechbrain.nnet.schedulers as schedulers
from speechbrain.core import AMPConfig
from speechbrain.utils.distributed import run_on_main
from speechbrain.utils.logger import get_logger
import time
from torch.utils.data import DataLoader

import musdb


# Define training procedure
class Separation(sb.Brain):
    def compute_forward(self, mix, targets, stage, noise=None):
        """Forward computations from the mixture to the separated signals."""

        # Unpack lists and put tensors in the right device
        mix, mix_lens = mix       
        mix, mix_lens = mix.to(self.device), mix_lens.to(self.device)      

        # Convert targets to tensor
        targets = torch.cat(
            [targets[i][0].unsqueeze(-1) for i in range(self.hparams.num_sources)],
            dim=-1,
        ).to(self.device)
        
        # Separation
        mix_w = self.hparams.Encoder(mix)
        est_mask = self.hparams.MaskNet(mix_w)
        mix_w = torch.stack([mix_w] * self.hparams.num_sources)
        sep_h = mix_w * est_mask
        
        # Decoding
        est_source = torch.cat(
            [
                self.hparams.Decoder(sep_h[i]).unsqueeze(-1)
                for i in range(self.hparams.num_sources)
            ],
            dim=-1,
        )

        # pad estimates as per requirement 
        T_origin = mix.size(1)
        T_est = est_source.size(1)
        if T_origin > T_est:
            est_source = F.pad(est_source, (0, 0, 0, T_origin - T_est))
        else:
            est_source = est_source[:, :T_origin, :]

        return est_source, targets

    def compute_objectives(self, predictions, targets):
        """Computes the sinr loss"""
        return self.hparams.loss(targets, predictions)

    def fit_batch(self, batch):
        """Trains one batch"""
        # print("INSIDE FIT BATCH")
        
        amp = AMPConfig.from_name(self.precision)
        should_step = (self.step % self.grad_accumulation_factor) == 0
        # Unpacking batch list
        mixture = batch.mix_sig
        targets = [batch.voc_sig, batch.inst_sig] #mix_sig, voc_sig, inst_sig
       
        
        predictions, targets = self.compute_forward(
            mixture, targets, sb.Stage.TRAIN
        )
        loss = self.compute_objectives(predictions, targets)

        if self.hparams.threshold_byloss:
            th = self.hparams.threshold
            loss = loss[loss > th]
            if loss.nelement() > 0:
                loss = loss.mean()
        else:
            loss = loss.mean()

        if (
            loss.nelement() > 0 and loss < self.hparams.loss_upper_lim
        ):  # the fix for computational problems
            loss.backward()
            if self.hparams.clip_grad_norm >= 0:
                torch.nn.utils.clip_grad_norm_(
                    self.modules.parameters(),
                    self.hparams.clip_grad_norm,
                )
            self.optimizer.step()
        else:
            self.nonfinite_count += 1
            logger.info(
                "infinite loss or empty loss! it happened {} times so far - skipping this batch".format(
                    self.nonfinite_count
                )
            )
            loss.data = torch.tensor(0.0).to(self.device)
        self.optimizer.zero_grad()

        return loss.detach().cpu()

    def evaluate_batch(self, batch, stage):
        """Computations needed for validation/test batches"""
        snt_id = batch.track_id
        mixture = batch.mix_sig
        targets = [batch.voc_sig, batch.inst_sig]

        with torch.no_grad():
            predictions, targets = self.compute_forward(mixture, targets, stage)
            loss = self.compute_objectives(predictions, targets)

        # Manage audio file saving
        if stage == sb.Stage.TEST and self.hparams.save_audio:
            if hasattr(self.hparams, "n_audio_to_save"):
                if self.hparams.n_audio_to_save > 0:
                    self.save_audio(snt_id[0], mixture, targets, predictions)
                    self.hparams.n_audio_to_save += -1
            else:
                self.save_audio(snt_id[0], mixture, targets, predictions)

        return loss.mean().detach()

    def on_stage_end(self, stage, stage_loss, epoch):
        """Gets called at the end of a epoch."""
        # Compute/store important stats
        stage_stats = {"si-snr": stage_loss}
        if stage == sb.Stage.TRAIN:
            self.train_stats = stage_stats

        # Perform end-of-iteration things, like annealing, logging, etc.
        if stage == sb.Stage.VALID:
            # Learning rate annealing
            if isinstance(
                self.hparams.lr_scheduler, schedulers.ReduceLROnPlateau
            ):
                current_lr, next_lr = self.hparams.lr_scheduler(
                    [self.optimizer], epoch, stage_loss
                )
                schedulers.update_learning_rate(self.optimizer, next_lr)
            else:
                # if we do not use the reducelronplateau, we do not change the lr
                current_lr = self.hparams.optimizer.optim.param_groups[0]["lr"]

            self.hparams.train_logger.log_stats(
                stats_meta={"epoch": epoch, "lr": current_lr},
                train_stats=self.train_stats,
                valid_stats=stage_stats,
            )
            self.checkpointer.save_and_keep_only(
                meta={"si-snr": stage_stats["si-snr"]}, min_keys=["si-snr"]
            )
        elif stage == sb.Stage.TEST:
            self.hparams.train_logger.log_stats(
                stats_meta={"Epoch loaded": self.hparams.epoch_counter.current},
                test_stats=stage_stats,
            )


    def cut_signals(self, mixture, targets):
        """This function selects a random segment of a given length within the mixture.
        The corresponding targets are selected accordingly"""
        randstart = torch.randint(
            0,
            1 + max(0, mixture.shape[1] - self.hparams.training_signal_len),
            (1,),
        ).item()
        targets = targets[
            :, randstart : randstart + self.hparams.training_signal_len, :
        ]
        mixture = mixture[
            :, randstart : randstart + self.hparams.training_signal_len
        ]
        return mixture, targets

    def reset_layer_recursively(self, layer):
        """Reinitializes the parameters of the neural networks"""
        if hasattr(layer, "reset_parameters"):
            layer.reset_parameters()
        for child_layer in layer.modules():
            if layer != child_layer:
                self.reset_layer_recursively(child_layer)

    def save_results(self, test_loader):
        """This script computes the SDR and SI-SNR metrics and saves
        them into a csv file"""

        # This package is required for SDR computation
        from mir_eval.separation import bss_eval_sources

        # Create folders where to store audio
        save_file = os.path.join(self.hparams.output_folder, "test_results.csv")

        # Variable init
        all_sdrs = []
        all_sdrs_i = []
        all_sisnrs = []
        all_sisnrs_i = []
        csv_columns = ["snt_id", "sdr", "sdr_i", "si-snr", "si-snr_i"]


        def is_silent(source, threshold=1e-6):
            return np.max(np.abs(source[0])) < threshold or np.max(np.abs(source[1])) < threshold

        with open(save_file, "w", newline="", encoding="utf-8") as results_csv:
            writer = csv.DictWriter(results_csv, fieldnames=csv_columns)
            writer.writeheader()
            skip_cnt = 0

            # Loop over all test sentence
            with tqdm(test_loader, dynamic_ncols=True) as t:
                for i, batch in enumerate(t):
                    # Apply Separation
                    mixture, mix_len = batch.mix_sig
                    snt_id = batch.track_id
                    targets = [batch.voc_sig, batch.inst_sig]
                   

                    with torch.no_grad():
                        predictions, targets = self.compute_forward(
                            batch.mix_sig, targets, sb.Stage.TEST
                        )

                    # Compute SI-SNR
                    sisnr = self.compute_objectives(predictions, targets)

                    # Compute SI-SNR improvement
                    mixture_signal = torch.stack(
                        [mixture] * self.hparams.num_sources, dim=-1
                    )
                    mixture_signal = mixture_signal.to(targets.device)
                    sisnr_baseline = self.compute_objectives(
                        mixture_signal, targets
                    )
                    sisnr_i = sisnr - sisnr_baseline
                    
     
                    if not is_silent(targets[0].t().cpu().numpy()) and not is_silent(predictions[0].t().detach().cpu().numpy()) and not is_silent(mixture_signal[0].t().detach().cpu().numpy()):
                        
                    
                        sdr, _, _, _ = bss_eval_sources(
                            targets[0].t().cpu().numpy(),
                            predictions[0].t().detach().cpu().numpy(),
                            compute_permutation=False
                        )
    
                        sdr_baseline, _, _, _ = bss_eval_sources(
                            targets[0].t().cpu().numpy(),
                            mixture_signal[0].t().detach().cpu().numpy(),
                            compute_permutation=False
                        )
    
                        sdr_i = sdr.mean() - sdr_baseline.mean()
    
                        # Saving on a csv file
                        row = {
                            "snt_id": snt_id[0],
                            "sdr": sdr.mean(),
                            "sdr_i": sdr_i,
                            "si-snr": -sisnr.item(),
                            "si-snr_i": -sisnr_i.item(),
                        }
                        writer.writerow(row)
    
                        # Metric Accumulation
                        all_sdrs.append(sdr.mean())
                        all_sdrs_i.append(sdr_i.mean())
                        all_sisnrs.append(-sisnr.item())
                        all_sisnrs_i.append(-sisnr_i.item())
    
                
                    else:
                        skip_cnt += 1
                        print(f"Warning: skipping silent target, this has happened {skip_cnt} times")

                row = {
                    "snt_id": "avg",
                    "sdr": np.array(all_sdrs).mean(),
                    "sdr_i": np.array(all_sdrs_i).mean(),
                    "si-snr": np.array(all_sisnrs).mean(),
                    "si-snr_i": np.array(all_sisnrs_i).mean(),
                }
                writer.writerow(row)
                
        logger.info("Mean SISNR is {}".format(np.array(all_sisnrs).mean()))
        logger.info("Mean SISNRi is {}".format(np.array(all_sisnrs_i).mean()))
        logger.info("Mean SDR is {}".format(np.array(all_sdrs).mean()))
        logger.info("Mean SDRi is {}".format(np.array(all_sdrs_i).mean()))
        if(self.hparams.result_file_path != ""):
            with open(self.hparams.result_file_path, "a", newline="", encoding="utf-8") as metrics_csv:
                writer = csv.DictWriter(metrics_csv, fieldnames=["model_name", "n_epochs", "learning_rate", "chunk_size", "sample_rate", "sdr", "sdr_i", "si-snr", "si-snr_i"])
                row = {
                        "model_name": self.hparams.experiment_name,
                        "learning_rate": self.hparams.lr,
                        "n_epochs": self.hparams.N_epochs,
                        "chunk_size":self.hparams.chunk_size,
                        "sample_rate":self.hparams.sample_rate,
                        "sdr": np.array(all_sdrs).mean(),
                        "sdr_i": np.array(all_sdrs_i).mean(),
                        "si-snr": np.array(all_sisnrs).mean(),
                        "si-snr_i": np.array(all_sisnrs_i).mean(),
                    }
                writer.writerow(row)

    def save_audio(self, snt_id, mixture, targets, predictions):
        "saves the test audio (mixture, targets, and estimated sources) on disk"

        # Create output folder
        save_path = os.path.join(self.hparams.save_folder, "audio_results")
        if not os.path.exists(save_path):
            os.mkdir(save_path)

        for ns in range(self.hparams.num_sources):
            # Estimated source
            signal = predictions[0, :, ns]
            signal = signal / signal.abs().max()
            save_file = os.path.join(
                save_path, "item{}_source{}hat.wav".format(snt_id, ns + 1)
            )
            torchaudio.save(
                save_file, signal.unsqueeze(0).cpu(), self.hparams.sample_rate
            )

            # Original source
            signal = targets[0, :, ns]
            signal = signal / signal.abs().max()
            save_file = os.path.join(
                save_path, "item{}_source{}.wav".format(snt_id, ns + 1)
            )
            torchaudio.save(
                save_file, signal.unsqueeze(0).cpu(), self.hparams.sample_rate
            )

        # Mixture
        signal = mixture[0][0, :]
        signal = signal / signal.abs().max()
        save_file = os.path.join(save_path, "item{}_mix.wav".format(snt_id))
        torchaudio.save(
            save_file, signal.unsqueeze(0).cpu(), self.hparams.sample_rate
        )





if __name__ == "__main__":
    # Load hyperparameters file with command-line overrides
    hparams_file, run_opts, overrides = sb.parse_arguments(sys.argv[1:])
    with open(hparams_file, encoding="utf-8") as fin:
        hparams = load_hyperpyyaml(fin, overrides)

    # Initialize ddp (useful only for multi-GPU DDP training)
    sb.utils.distributed.ddp_init_group(run_opts)

    # Logger info
    logger = get_logger(__name__)

    # Create experiment directory
    sb.create_experiment_directory(
        experiment_directory=hparams["output_folder"],
        hyperparams_to_save=hparams_file,
        overrides=overrides,
    )

    # Update precision to bf16 if the device is CPU and precision is fp16
    if run_opts.get("device") == "cpu" and hparams.get("precision") == "fp16":
        hparams["precision"] = "bf16"

   

    # Brain class initialization
    separator = Separation(
        modules=hparams["modules"],
        opt_class=hparams["optimizer"],
        hparams=hparams,
        run_opts=run_opts,
        checkpointer=hparams["checkpointer"],
    )
 
    # Training
        
    # Usage with SpeechBrain
    train_data = MusDBDataset(hparams["db_path"], subset="train", split="train", target_sr=hparams["sample_rate"], chunk_size=hparams["chunk_size"])
    valid_data = MusDBDataset(hparams["db_path"], subset="train", split="valid", target_sr=hparams["sample_rate"], chunk_size=hparams["chunk_size"])
    test_data = MusDBDataset(hparams["db_path"], subset="test", target_sr=hparams["sample_rate"], chunk_size=hparams["chunk_size"])
    

    # Create DataLoader
    train_loader = sb.dataio.dataloader.make_dataloader(
        train_data,
        batch_size=hparams["batch_size"],
        collate_fn=sb.dataio.batch.PaddedBatch  # Handles variable lengths
    )

    valid_loader = sb.dataio.dataloader.make_dataloader(
        valid_data,
        batch_size=hparams["batch_size"],
        collate_fn=sb.dataio.batch.PaddedBatch  # Handles variable lengths
    )

    test_loader = sb.dataio.dataloader.make_dataloader(
        test_data,
        batch_size=hparams["batch_size"],
        collate_fn=sb.dataio.batch.PaddedBatch  # Handles variable lengths
    )
    
    separator.fit(
        separator.hparams.epoch_counter,
        train_loader,
        valid_loader,
        train_loader_kwargs=hparams["dataloader_opts"],
        valid_loader_kwargs=hparams["dataloader_opts"],
    )

    # # Eval
    separator.evaluate(test_loader, min_key="si-snr")
    separator.save_results(test_loader)

Overwriting convtasnet-train.py


In [36]:
!python3 convtasnet-train.py convtasnet-hparams.yaml --data_folder=db_path --result_file_path={result_file_path}

speechbrain.utils.quirks - Applied quirks (see `speechbrain.utils.quirks`): [disable_jit_profiling, allow_tf32]
speechbrain.utils.quirks - Excluded quirks specified by the `SB_DISABLE_QUIRKS` environment (comma-separated list): []
speechbrain.core - Beginning experiment!
speechbrain.core - Experiment folder: /notebooks/results/convtasnet/1234
speechbrain.core - Info: precision arg from hparam file is used
speechbrain.core - Info: noprogressbar arg from hparam file is used
speechbrain.core - Gradscaler enabled: `True`
speechbrain.core - Using training precision: `--precision=fp16`
speechbrain.core - Using evaluation precision: `--eval_precision=fp32`
speechbrain.core - Separation Model Statistics:
* Total Number of Trainable Parameters: 6.6M
* Total Number of Parameters: 6.6M
* Trainable Parameters represent 100.0000% of the total size.
speechbrain.utils.checkpoints - Loading a checkpoint from /notebooks/results/convtasnet/1234/save/CKPT+2025-04-17+11-37-49+00
speechbrain.utils.checkpoi

## DPRNN


In [5]:
%%file dprnn-hparams.yaml
# ################################
# Model: DPRNN for Music vocal separation
# https://arxiv.org/abs/2010.13154
# Dataset : MusDB
# ################################
# Basic parameters
# Seed needs to be set at top of yaml, before objects with parameters are made


seed: 1234
__set_seed: !apply:speechbrain.utils.seed_everything [!ref <seed>]

# Data params

data_folder: !PLACEHOLDER
result_file_path: !PLACEHOLDER

experiment_name: dprnn
output_folder: !ref /notebooks/results/<experiment_name>/<seed>
train_log: !ref <output_folder>/train_log.txt
save_folder: !ref <output_folder>/save
train_data: !ref <output_folder>/train.json
valid_data: !ref <output_folder>/valid.json
test_data: !ref <output_folder>/test.json
skip_prep: False
db_path: '/notebooks/data'


# Experiment params
precision: fp16 # bf16, fp16 or fp32

instrumental_classification: False
noprogressbar: False
save_audio: True # Save estimated sources on disk
sample_rate: 8000
n_audio_to_save: 10
chunk_size: 20

####################### Training Parameters ####################################
N_epochs: 30
batch_size: 1
lr: 0.00015
clip_grad_norm: 5
loss_upper_lim: 999999  # this is the upper limit for an acceptable loss
num_sources: 2



# loss thresholding -- this thresholds the training loss
threshold_byloss: True
threshold: -30

# Encoder parameters
N_encoder_out: 256
out_channels: 256
kernel_size: 16
kernel_stride: 8

# Dataloader options
dataloader_opts:
    batch_size: !ref <batch_size>
    num_workers: 3


# Specifying the network
Encoder: !new:speechbrain.lobes.models.dual_path.Encoder
    kernel_size: !ref <kernel_size>
    out_channels: !ref <N_encoder_out>

intra: !new:speechbrain.lobes.models.dual_path.SBRNNBlock
    num_layers: 1
    input_size: !ref <out_channels>
    hidden_channels: !ref <out_channels>
    dropout: 0
    bidirectional: True

inter: !new:speechbrain.lobes.models.dual_path.SBRNNBlock
    num_layers: 1
    input_size: !ref <out_channels>
    hidden_channels: !ref <out_channels>
    dropout: 0
    bidirectional: True

MaskNet: !new:speechbrain.lobes.models.dual_path.Dual_Path_Model
    num_spks: !ref <num_sources>
    in_channels: !ref <N_encoder_out>
    out_channels: !ref <out_channels>
    num_layers: 6
    K: 250
    intra_model: !ref <intra>
    inter_model: !ref <inter>
    norm: ln
    linear_layer_after_inter_intra: True
    skip_around_intra: True

Decoder: !new:speechbrain.lobes.models.dual_path.Decoder
    in_channels: !ref <N_encoder_out>
    out_channels: 1
    kernel_size: !ref <kernel_size>
    stride: !ref <kernel_stride>
    bias: False

optimizer: !name:torch.optim.Adam
    lr: !ref <lr>
    weight_decay: 0

loss: !name:speechbrain.nnet.losses.get_si_snr_with_pitwrapper

lr_scheduler: !new:speechbrain.nnet.schedulers.ReduceLROnPlateau
    factor: 0.5
    patience: 2
    dont_halve_until_epoch: 85

epoch_counter: !new:speechbrain.utils.epoch_loop.EpochCounter
    limit: !ref <N_epochs>

modules:
    encoder: !ref <Encoder>
    decoder: !ref <Decoder>
    masknet: !ref <MaskNet>

checkpointer: !new:speechbrain.utils.checkpoints.Checkpointer
    checkpoints_dir: !ref <save_folder>
    recoverables:
        encoder: !ref <Encoder>
        decoder: !ref <Decoder>
        masknet: !ref <MaskNet>
        counter: !ref <epoch_counter>
        lr_scheduler: !ref <lr_scheduler>

train_logger: !new:speechbrain.utils.train_logger.FileTrainLogger
    save_file: !ref <train_log>

Overwriting dprnn-hparams.yaml


In [6]:
%%file dprnn-train.py
#!/usr/bin/env/python3
"""
Recipe for vocal separation using convtasnet
"""

import csv
import os
import sys

import numpy as np
import torch
import torch.nn.functional as F
import torchaudio
from hyperpyyaml import load_hyperpyyaml
from tqdm import tqdm
import pdb

import musdb
import torchaudio
import numpy as np
from torch.utils.data import Dataset
import speechbrain as sb
import psutil
from dataset import MusDBDataset


import speechbrain as sb
import speechbrain.nnet.schedulers as schedulers
from speechbrain.core import AMPConfig
from speechbrain.utils.distributed import run_on_main
from speechbrain.utils.logger import get_logger
import time
from torch.utils.data import DataLoader

import musdb


# Define training procedure
class Separation(sb.Brain):
    def compute_forward(self, mix, targets, stage, noise=None):
        """Forward computations from the mixture to the separated signals."""

        # Unpack lists and put tensors in the right device
        mix, mix_lens = mix       
        mix, mix_lens = mix.to(self.device), mix_lens.to(self.device)      

        # Convert targets to tensor
        targets = torch.cat(
            [targets[i][0].unsqueeze(-1) for i in range(self.hparams.num_sources)],
            dim=-1,
        ).to(self.device)
        
        # Separation
        mix_w = self.hparams.Encoder(mix)
        est_mask = self.hparams.MaskNet(mix_w)
        mix_w = torch.stack([mix_w] * self.hparams.num_sources)
        sep_h = mix_w * est_mask
        
        # Decoding
        est_source = torch.cat(
            [
                self.hparams.Decoder(sep_h[i]).unsqueeze(-1)
                for i in range(self.hparams.num_sources)
            ],
            dim=-1,
        )

        # pad estimates as per requirement 
        T_origin = mix.size(1)
        T_est = est_source.size(1)
        if T_origin > T_est:
            est_source = F.pad(est_source, (0, 0, 0, T_origin - T_est))
        else:
            est_source = est_source[:, :T_origin, :]

        return est_source, targets

    def compute_objectives(self, predictions, targets):
        """Computes the sinr loss"""
        return self.hparams.loss(targets, predictions)

    def fit_batch(self, batch):
        """Trains one batch"""
        # print("INSIDE FIT BATCH")
        
        amp = AMPConfig.from_name(self.precision)
        should_step = (self.step % self.grad_accumulation_factor) == 0
        # Unpacking batch list
        mixture = batch.mix_sig
        targets = [batch.voc_sig, batch.inst_sig] #mix_sig, voc_sig, inst_sig
       
        
        predictions, targets = self.compute_forward(
            mixture, targets, sb.Stage.TRAIN
        )
        loss = self.compute_objectives(predictions, targets)

        if self.hparams.threshold_byloss:
            th = self.hparams.threshold
            loss = loss[loss > th]
            if loss.nelement() > 0:
                loss = loss.mean()
        else:
            loss = loss.mean()

        if (
            loss.nelement() > 0 and loss < self.hparams.loss_upper_lim
        ):  # the fix for computational problems
            loss.backward()
            if self.hparams.clip_grad_norm >= 0:
                torch.nn.utils.clip_grad_norm_(
                    self.modules.parameters(),
                    self.hparams.clip_grad_norm,
                )
            self.optimizer.step()
        else:
            self.nonfinite_count += 1
            logger.info(
                "infinite loss or empty loss! it happened {} times so far - skipping this batch".format(
                    self.nonfinite_count
                )
            )
            loss.data = torch.tensor(0.0).to(self.device)
        self.optimizer.zero_grad()

        return loss.detach().cpu()

    def evaluate_batch(self, batch, stage):
        """Computations needed for validation/test batches"""
        snt_id = batch.track_id
        mixture = batch.mix_sig
        targets = [batch.voc_sig, batch.inst_sig]

        with torch.no_grad():
            predictions, targets = self.compute_forward(mixture, targets, stage)
            loss = self.compute_objectives(predictions, targets)

        # Manage audio file saving
        if stage == sb.Stage.TEST and self.hparams.save_audio:
            if hasattr(self.hparams, "n_audio_to_save"):
                if self.hparams.n_audio_to_save > 0:
                    self.save_audio(snt_id[0], mixture, targets, predictions)
                    self.hparams.n_audio_to_save += -1
            else:
                self.save_audio(snt_id[0], mixture, targets, predictions)

        return loss.mean().detach()

    def on_stage_end(self, stage, stage_loss, epoch):
        """Gets called at the end of a epoch."""
        # Compute/store important stats
        stage_stats = {"si-snr": stage_loss}
        if stage == sb.Stage.TRAIN:
            self.train_stats = stage_stats

        # Perform end-of-iteration things, like annealing, logging, etc.
        if stage == sb.Stage.VALID:
            # Learning rate annealing
            if isinstance(
                self.hparams.lr_scheduler, schedulers.ReduceLROnPlateau
            ):
                current_lr, next_lr = self.hparams.lr_scheduler(
                    [self.optimizer], epoch, stage_loss
                )
                schedulers.update_learning_rate(self.optimizer, next_lr)
            else:
                # if we do not use the reducelronplateau, we do not change the lr
                current_lr = self.hparams.optimizer.optim.param_groups[0]["lr"]

            self.hparams.train_logger.log_stats(
                stats_meta={"epoch": epoch, "lr": current_lr},
                train_stats=self.train_stats,
                valid_stats=stage_stats,
            )
            self.checkpointer.save_and_keep_only(
                meta={"si-snr": stage_stats["si-snr"]}, min_keys=["si-snr"]
            )
        elif stage == sb.Stage.TEST:
            self.hparams.train_logger.log_stats(
                stats_meta={"Epoch loaded": self.hparams.epoch_counter.current},
                test_stats=stage_stats,
            )


    def cut_signals(self, mixture, targets):
        """This function selects a random segment of a given length within the mixture.
        The corresponding targets are selected accordingly"""
        randstart = torch.randint(
            0,
            1 + max(0, mixture.shape[1] - self.hparams.training_signal_len),
            (1,),
        ).item()
        targets = targets[
            :, randstart : randstart + self.hparams.training_signal_len, :
        ]
        mixture = mixture[
            :, randstart : randstart + self.hparams.training_signal_len
        ]
        return mixture, targets

    def reset_layer_recursively(self, layer):
        """Reinitializes the parameters of the neural networks"""
        if hasattr(layer, "reset_parameters"):
            layer.reset_parameters()
        for child_layer in layer.modules():
            if layer != child_layer:
                self.reset_layer_recursively(child_layer)

    def save_results(self, test_loader):
        """This script computes the SDR and SI-SNR metrics and saves
        them into a csv file"""

        # This package is required for SDR computation
        from mir_eval.separation import bss_eval_sources

        # Create folders where to store audio
        save_file = os.path.join(self.hparams.output_folder, "test_results.csv")

        # Variable init
        all_sdrs = []
        all_sdrs_i = []
        all_sisnrs = []
        all_sisnrs_i = []
        csv_columns = ["snt_id", "sdr", "sdr_i", "si-snr", "si-snr_i"]


        def is_silent(source, threshold=1e-6):
            return np.max(np.abs(source[0])) < threshold or np.max(np.abs(source[1])) < threshold

        with open(save_file, "w", newline="", encoding="utf-8") as results_csv:
            writer = csv.DictWriter(results_csv, fieldnames=csv_columns)
            writer.writeheader()
            skip_cnt = 0

            # Loop over all test sentence
            with tqdm(test_loader, dynamic_ncols=True) as t:
                for i, batch in enumerate(t):
                    # Apply Separation
                    mixture, mix_len = batch.mix_sig
                    snt_id = batch.track_id
                    targets = [batch.voc_sig, batch.inst_sig]
                   

                    with torch.no_grad():
                        predictions, targets = self.compute_forward(
                            batch.mix_sig, targets, sb.Stage.TEST
                        )

                    # Compute SI-SNR
                    sisnr = self.compute_objectives(predictions, targets)

                    # Compute SI-SNR improvement
                    mixture_signal = torch.stack(
                        [mixture] * self.hparams.num_sources, dim=-1
                    )
                    mixture_signal = mixture_signal.to(targets.device)
                    sisnr_baseline = self.compute_objectives(
                        mixture_signal, targets
                    )
                    sisnr_i = sisnr - sisnr_baseline
                    
     
                    if not is_silent(targets[0].t().cpu().numpy()) and not is_silent(predictions[0].t().detach().cpu().numpy()) and not is_silent(mixture_signal[0].t().detach().cpu().numpy()):
                        
                    
                        sdr, _, _, _ = bss_eval_sources(
                            targets[0].t().cpu().numpy(),
                            predictions[0].t().detach().cpu().numpy(),
                            compute_permutation=False
                        )
    
                        sdr_baseline, _, _, _ = bss_eval_sources(
                            targets[0].t().cpu().numpy(),
                            mixture_signal[0].t().detach().cpu().numpy(),
                            compute_permutation=False
                        )
    
                        sdr_i = sdr.mean() - sdr_baseline.mean()
    
                        # Saving on a csv file
                        row = {
                            "snt_id": snt_id[0],
                            "sdr": sdr.mean(),
                            "sdr_i": sdr_i,
                            "si-snr": -sisnr.item(),
                            "si-snr_i": -sisnr_i.item(),
                        }
                        writer.writerow(row)
    
                        # Metric Accumulation
                        all_sdrs.append(sdr.mean())
                        all_sdrs_i.append(sdr_i.mean())
                        all_sisnrs.append(-sisnr.item())
                        all_sisnrs_i.append(-sisnr_i.item())
                    else:
                        skip_cnt += 1
                        print(f"Warning: skipping silent target, this has happened {skip_cnt} times")
            row = {
                "snt_id": "avg",
                "sdr": np.array(all_sdrs).mean(),
                "sdr_i": np.array(all_sdrs_i).mean(),
                "si-snr": np.array(all_sisnrs).mean(),
                "si-snr_i": np.array(all_sisnrs_i).mean(),
            }
            writer.writerow(row)

        logger.info("Mean SISNR is {}".format(np.array(all_sisnrs).mean()))
        logger.info("Mean SISNRi is {}".format(np.array(all_sisnrs_i).mean()))
        logger.info("Mean SDR is {}".format(np.array(all_sdrs).mean()))
        logger.info("Mean SDRi is {}".format(np.array(all_sdrs_i).mean()))
        if(self.hparams.result_file_path != ""):
            with open(self.hparams.result_file_path, "a", newline="", encoding="utf-8") as metrics_csv:
                writer = csv.DictWriter(metrics_csv, fieldnames=["model_name", "n_epochs", "learning_rate", "chunk_size", "sample_rate", "sdr", "sdr_i", "si-snr", "si-snr_i"])
                row = {
                        "model_name": self.hparams.experiment_name,
                        "learning_rate": self.hparams.lr,
                        "n_epochs": self.hparams.N_epochs,
                        "chunk_size":self.hparams.chunk_size,
                        "sample_rate":self.hparams.sample_rate,
                        "sdr": np.array(all_sdrs).mean(),
                        "sdr_i": np.array(all_sdrs_i).mean(),
                        "si-snr": np.array(all_sisnrs).mean(),
                        "si-snr_i": np.array(all_sisnrs_i).mean(),
                    }
                writer.writerow(row)

    def save_audio(self, snt_id, mixture, targets, predictions):
        "saves the test audio (mixture, targets, and estimated sources) on disk"

        # Create output folder
        save_path = os.path.join(self.hparams.save_folder, "audio_results")
        if not os.path.exists(save_path):
            os.mkdir(save_path)

        for ns in range(self.hparams.num_sources):
            # Estimated source
            signal = predictions[0, :, ns]
            signal = signal / signal.abs().max()
            save_file = os.path.join(
                save_path, "item{}_source{}hat.wav".format(snt_id, ns + 1)
            )
            torchaudio.save(
                save_file, signal.unsqueeze(0).cpu(), self.hparams.sample_rate
            )

            # Original source
            signal = targets[0, :, ns]
            signal = signal / signal.abs().max()
            save_file = os.path.join(
                save_path, "item{}_source{}.wav".format(snt_id, ns + 1)
            )
            torchaudio.save(
                save_file, signal.unsqueeze(0).cpu(), self.hparams.sample_rate
            )

        # Mixture
        signal = mixture[0][0, :]
        signal = signal / signal.abs().max()
        save_file = os.path.join(save_path, "item{}_mix.wav".format(snt_id))
        torchaudio.save(
            save_file, signal.unsqueeze(0).cpu(), self.hparams.sample_rate
        )





if __name__ == "__main__":
    # Load hyperparameters file with command-line overrides
    hparams_file, run_opts, overrides = sb.parse_arguments(sys.argv[1:])
    with open(hparams_file, encoding="utf-8") as fin:
        hparams = load_hyperpyyaml(fin, overrides)

    # Initialize ddp (useful only for multi-GPU DDP training)
    sb.utils.distributed.ddp_init_group(run_opts)

    # Logger info
    logger = get_logger(__name__)

    # Create experiment directory
    sb.create_experiment_directory(
        experiment_directory=hparams["output_folder"],
        hyperparams_to_save=hparams_file,
        overrides=overrides,
    )

    # Update precision to bf16 if the device is CPU and precision is fp16
    if run_opts.get("device") == "cpu" and hparams.get("precision") == "fp16":
        hparams["precision"] = "bf16"

   

    # Brain class initialization
    separator = Separation(
        modules=hparams["modules"],
        opt_class=hparams["optimizer"],
        hparams=hparams,
        run_opts=run_opts,
        checkpointer=hparams["checkpointer"],
    )
 
    # Training
        
    # Usage with SpeechBrain
    train_data = MusDBDataset(hparams["db_path"], subset="train", split="train", target_sr=hparams["sample_rate"], chunk_size=hparams["chunk_size"])
    valid_data = MusDBDataset(hparams["db_path"], subset="train", split="valid", target_sr=hparams["sample_rate"], chunk_size=hparams["chunk_size"])
    test_data = MusDBDataset(hparams["db_path"], subset="test", target_sr=hparams["sample_rate"], chunk_size=hparams["chunk_size"])
    

    # Create DataLoader
    train_loader = sb.dataio.dataloader.make_dataloader(
        train_data,
        batch_size=hparams["batch_size"],
        collate_fn=sb.dataio.batch.PaddedBatch  # Handles variable lengths
    )

    valid_loader = sb.dataio.dataloader.make_dataloader(
        valid_data,
        batch_size=hparams["batch_size"],
        collate_fn=sb.dataio.batch.PaddedBatch  # Handles variable lengths
    )

    test_loader = sb.dataio.dataloader.make_dataloader(
        test_data,
        batch_size=hparams["batch_size"],
        collate_fn=sb.dataio.batch.PaddedBatch  # Handles variable lengths
    )
    # print("STARTING FIT")
    separator.fit(
        separator.hparams.epoch_counter,
        train_loader,
        valid_loader,
        train_loader_kwargs=hparams["dataloader_opts"],
        valid_loader_kwargs=hparams["dataloader_opts"],
    )

    # # Eval
    separator.evaluate(test_loader, min_key="si-snr")
    separator.save_results(test_loader)

Overwriting dprnn-train.py


In [7]:
!python3 dprnn-train.py dprnn-hparams.yaml --data_folder=db_path --result_file_path={result_file_path}

speechbrain.utils.quirks - Applied quirks (see `speechbrain.utils.quirks`): [disable_jit_profiling, allow_tf32]
speechbrain.utils.quirks - Excluded quirks specified by the `SB_DISABLE_QUIRKS` environment (comma-separated list): []
speechbrain.core - Beginning experiment!
speechbrain.core - Experiment folder: /notebooks/results/dprnn/1234
speechbrain.core - Info: precision arg from hparam file is used
speechbrain.core - Info: noprogressbar arg from hparam file is used
speechbrain.core - Gradscaler enabled: `True`
speechbrain.core - Using training precision: `--precision=fp16`
speechbrain.core - Using evaluation precision: `--eval_precision=fp32`
speechbrain.core - Separation Model Statistics:
* Total Number of Trainable Parameters: 14.6M
* Total Number of Parameters: 14.6M
* Trainable Parameters represent 100.0000% of the total size.
speechbrain.utils.checkpoints - Loading a checkpoint from /notebooks/results/dprnn/1234/save/CKPT+2025-04-18+00-10-38+00
speechbrain.utils.checkpoints - Lo

## Sepformer


In [8]:
%%file sepformer-hparams.yaml
# ################################
# Model: DPRNN for Music vocal separation
# https://arxiv.org/abs/2010.13154
# Dataset : MusDB
# ################################
# Basic parameters
# Seed needs to be set at top of yaml, before objects with parameters are made


seed: 1234
__set_seed: !apply:speechbrain.utils.seed_everything [!ref <seed>]

# Data params

data_folder: !PLACEHOLDER
result_file_path: !PLACEHOLDER


experiment_name: sepformer
output_folder: !ref /notebooks/results/<experiment_name>/<seed>
train_log: !ref <output_folder>/train_log.txt
save_folder: !ref <output_folder>/save
train_data: !ref <output_folder>/train.json
valid_data: !ref <output_folder>/valid.json
test_data: !ref <output_folder>/test.json
skip_prep: False
db_path: '/notebooks/data'


# Experiment params
precision: fp16 # bf16, fp16 or fp32

instrumental_classification: False
noprogressbar: False
save_audio: True # Save estimated sources on disk
sample_rate: 8000
n_audio_to_save: 10
chunk_size: 30

####################### Training Parameters ####################################
N_epochs: 30
batch_size: 1
lr: 0.00015
clip_grad_norm: 5
loss_upper_lim: 999999  # this is the upper limit for an acceptable loss
num_sources: 2



# loss thresholding -- this thresholds the training loss
threshold_byloss: True
threshold: -30

# Encoder parameters
N_encoder_out: 256
out_channels: 256
kernel_size: 16
kernel_stride: 8

# Dataloader options
# Set num_workers: 0 on MacOS due to behavior of the multiprocessing library
dataloader_opts:
    batch_size: !ref <batch_size>
    num_workers: 3


# Specifying the network
Encoder: !new:speechbrain.lobes.models.dual_path.Encoder
    kernel_size: !ref <kernel_size>
    out_channels: !ref <N_encoder_out>


SBtfintra: !new:speechbrain.lobes.models.dual_path.SBTransformerBlock
    num_layers: 2
    d_model: !ref <out_channels>
    nhead: 4
    d_ffn: 1024
    dropout: 0
    use_positional_encoding: True
    norm_before: True

SBtfinter: !new:speechbrain.lobes.models.dual_path.SBTransformerBlock
    num_layers: 2
    d_model: !ref <out_channels>
    nhead: 4
    d_ffn: 1024
    dropout: 0
    use_positional_encoding: True
    norm_before: True

MaskNet: !new:speechbrain.lobes.models.dual_path.Dual_Path_Model
    num_spks: !ref <num_sources>
    in_channels: !ref <N_encoder_out>
    out_channels: !ref <out_channels>
    num_layers: 2
    K: 250
    intra_model: !ref <SBtfintra>
    inter_model: !ref <SBtfinter>
    norm: ln
    linear_layer_after_inter_intra: False
    skip_around_intra: True

Decoder: !new:speechbrain.lobes.models.dual_path.Decoder
    in_channels: !ref <N_encoder_out>
    out_channels: 1
    kernel_size: !ref <kernel_size>
    stride: !ref <kernel_stride>
    bias: False

optimizer: !name:torch.optim.Adam
    lr: !ref <lr>
    weight_decay: 0

loss: !name:speechbrain.nnet.losses.get_si_snr_with_pitwrapper

lr_scheduler: !new:speechbrain.nnet.schedulers.ReduceLROnPlateau
    factor: 0.5
    patience: 2
    dont_halve_until_epoch: 85

epoch_counter: !new:speechbrain.utils.epoch_loop.EpochCounter
    limit: !ref <N_epochs>

modules:
    encoder: !ref <Encoder>
    decoder: !ref <Decoder>
    masknet: !ref <MaskNet>

checkpointer: !new:speechbrain.utils.checkpoints.Checkpointer
    checkpoints_dir: !ref <save_folder>
    recoverables:
        encoder: !ref <Encoder>
        decoder: !ref <Decoder>
        masknet: !ref <MaskNet>
        counter: !ref <epoch_counter>
        lr_scheduler: !ref <lr_scheduler>

train_logger: !new:speechbrain.utils.train_logger.FileTrainLogger
    save_file: !ref <train_log>

Overwriting sepformer-hparams.yaml


In [9]:
%%file sepformer-train.py
#!/usr/bin/env/python3
"""
Recipe for vocal separation using convtasnet
"""

import csv
import os
import sys

import numpy as np
import torch
import torch.nn.functional as F
import torchaudio
from hyperpyyaml import load_hyperpyyaml
from tqdm import tqdm
import pdb

import musdb
import torchaudio
import numpy as np
from torch.utils.data import Dataset
import speechbrain as sb
import psutil
from dataset import MusDBDataset


import speechbrain as sb
import speechbrain.nnet.schedulers as schedulers
from speechbrain.core import AMPConfig
from speechbrain.utils.distributed import run_on_main
from speechbrain.utils.logger import get_logger
import time
from torch.utils.data import DataLoader

import musdb


# Define training procedure
class Separation(sb.Brain):
    def compute_forward(self, mix, targets, stage, noise=None):
        """Forward computations from the mixture to the separated signals."""

        # Unpack lists and put tensors in the right device
        mix, mix_lens = mix       
        mix, mix_lens = mix.to(self.device), mix_lens.to(self.device)      

        # Convert targets to tensor
        targets = torch.cat(
            [targets[i][0].unsqueeze(-1) for i in range(self.hparams.num_sources)],
            dim=-1,
        ).to(self.device)
        
        # Separation
        mix_w = self.hparams.Encoder(mix)
        est_mask = self.hparams.MaskNet(mix_w)
        mix_w = torch.stack([mix_w] * self.hparams.num_sources)
        sep_h = mix_w * est_mask
        
        # Decoding
        est_source = torch.cat(
            [
                self.hparams.Decoder(sep_h[i]).unsqueeze(-1)
                for i in range(self.hparams.num_sources)
            ],
            dim=-1,
        )

        # pad estimates as per requirement 
        T_origin = mix.size(1)
        T_est = est_source.size(1)
        if T_origin > T_est:
            est_source = F.pad(est_source, (0, 0, 0, T_origin - T_est))
        else:
            est_source = est_source[:, :T_origin, :]

        return est_source, targets

    def compute_objectives(self, predictions, targets):
        """Computes the sinr loss"""
        return self.hparams.loss(targets, predictions)

    def fit_batch(self, batch):
        """Trains one batch"""
        # print("INSIDE FIT BATCH")
        
        amp = AMPConfig.from_name(self.precision)
        should_step = (self.step % self.grad_accumulation_factor) == 0
        # Unpacking batch list
        mixture = batch.mix_sig
        targets = [batch.voc_sig, batch.inst_sig] #mix_sig, voc_sig, inst_sig
       
        
        predictions, targets = self.compute_forward(
            mixture, targets, sb.Stage.TRAIN
        )
        loss = self.compute_objectives(predictions, targets)

        if self.hparams.threshold_byloss:
            th = self.hparams.threshold
            loss = loss[loss > th]
            if loss.nelement() > 0:
                loss = loss.mean()
        else:
            loss = loss.mean()

        if (
            loss.nelement() > 0 and loss < self.hparams.loss_upper_lim
        ):  # the fix for computational problems
            loss.backward()
            if self.hparams.clip_grad_norm >= 0:
                torch.nn.utils.clip_grad_norm_(
                    self.modules.parameters(),
                    self.hparams.clip_grad_norm,
                )
            self.optimizer.step()
        else:
            self.nonfinite_count += 1
            logger.info(
                "infinite loss or empty loss! it happened {} times so far - skipping this batch".format(
                    self.nonfinite_count
                )
            )
            loss.data = torch.tensor(0.0).to(self.device)
        self.optimizer.zero_grad()

        return loss.detach().cpu()

    def evaluate_batch(self, batch, stage):
        """Computations needed for validation/test batches"""
        snt_id = batch.track_id
        mixture = batch.mix_sig
        targets = [batch.voc_sig, batch.inst_sig]

        with torch.no_grad():
            predictions, targets = self.compute_forward(mixture, targets, stage)
            loss = self.compute_objectives(predictions, targets)

        # Manage audio file saving
        if stage == sb.Stage.TEST and self.hparams.save_audio:
            if hasattr(self.hparams, "n_audio_to_save"):
                if self.hparams.n_audio_to_save > 0:
                    self.save_audio(snt_id[0], mixture, targets, predictions)
                    self.hparams.n_audio_to_save += -1
            else:
                self.save_audio(snt_id[0], mixture, targets, predictions)

        return loss.mean().detach()

    def on_stage_end(self, stage, stage_loss, epoch):
        """Gets called at the end of a epoch."""
        # Compute/store important stats
        stage_stats = {"si-snr": stage_loss}
        if stage == sb.Stage.TRAIN:
            self.train_stats = stage_stats

        # Perform end-of-iteration things, like annealing, logging, etc.
        if stage == sb.Stage.VALID:
            # Learning rate annealing
            if isinstance(
                self.hparams.lr_scheduler, schedulers.ReduceLROnPlateau
            ):
                current_lr, next_lr = self.hparams.lr_scheduler(
                    [self.optimizer], epoch, stage_loss
                )
                schedulers.update_learning_rate(self.optimizer, next_lr)
            else:
                # if we do not use the reducelronplateau, we do not change the lr
                current_lr = self.hparams.optimizer.optim.param_groups[0]["lr"]

            self.hparams.train_logger.log_stats(
                stats_meta={"epoch": epoch, "lr": current_lr},
                train_stats=self.train_stats,
                valid_stats=stage_stats,
            )
            self.checkpointer.save_and_keep_only(
                meta={"si-snr": stage_stats["si-snr"]}, min_keys=["si-snr"]
            )
        elif stage == sb.Stage.TEST:
            self.hparams.train_logger.log_stats(
                stats_meta={"Epoch loaded": self.hparams.epoch_counter.current},
                test_stats=stage_stats,
            )


    def cut_signals(self, mixture, targets):
        """This function selects a random segment of a given length within the mixture.
        The corresponding targets are selected accordingly"""
        randstart = torch.randint(
            0,
            1 + max(0, mixture.shape[1] - self.hparams.training_signal_len),
            (1,),
        ).item()
        targets = targets[
            :, randstart : randstart + self.hparams.training_signal_len, :
        ]
        mixture = mixture[
            :, randstart : randstart + self.hparams.training_signal_len
        ]
        return mixture, targets

    def reset_layer_recursively(self, layer):
        """Reinitializes the parameters of the neural networks"""
        if hasattr(layer, "reset_parameters"):
            layer.reset_parameters()
        for child_layer in layer.modules():
            if layer != child_layer:
                self.reset_layer_recursively(child_layer)

    def save_results(self, test_loader):
        """This script computes the SDR and SI-SNR metrics and saves
        them into a csv file"""

        # This package is required for SDR computation
        from mir_eval.separation import bss_eval_sources

        # Create folders where to store audio
        save_file = os.path.join(self.hparams.output_folder, "test_results.csv")

        # Variable init
        all_sdrs = []
        all_sdrs_i = []
        all_sisnrs = []
        all_sisnrs_i = []
        csv_columns = ["snt_id", "sdr", "sdr_i", "si-snr", "si-snr_i"]


        def is_silent(source, threshold=1e-6):
            return np.max(np.abs(source[0])) < threshold or np.max(np.abs(source[1])) < threshold

        with open(save_file, "w", newline="", encoding="utf-8") as results_csv:
            writer = csv.DictWriter(results_csv, fieldnames=csv_columns)
            writer.writeheader()
            skip_cnt = 0

            # Loop over all test sentence
            with tqdm(test_loader, dynamic_ncols=True) as t:
                for i, batch in enumerate(t):
                    # Apply Separation
                    mixture, mix_len = batch.mix_sig
                    snt_id = batch.track_id
                    targets = [batch.voc_sig, batch.inst_sig]
                   

                    with torch.no_grad():
                        predictions, targets = self.compute_forward(
                            batch.mix_sig, targets, sb.Stage.TEST
                        )

                    # Compute SI-SNR
                    sisnr = self.compute_objectives(predictions, targets)

                    # Compute SI-SNR improvement
                    mixture_signal = torch.stack(
                        [mixture] * self.hparams.num_sources, dim=-1
                    )
                    mixture_signal = mixture_signal.to(targets.device)
                    sisnr_baseline = self.compute_objectives(
                        mixture_signal, targets
                    )
                    sisnr_i = sisnr - sisnr_baseline
                    
     
                    if not is_silent(targets[0].t().cpu().numpy()) and not is_silent(predictions[0].t().detach().cpu().numpy()) and not is_silent(mixture_signal[0].t().detach().cpu().numpy()):
                        
                    
                        sdr, _, _, _ = bss_eval_sources(
                            targets[0].t().cpu().numpy(),
                            predictions[0].t().detach().cpu().numpy(),
                            compute_permutation=False
                        )
    
                        sdr_baseline, _, _, _ = bss_eval_sources(
                            targets[0].t().cpu().numpy(),
                            mixture_signal[0].t().detach().cpu().numpy(),
                            compute_permutation=False
                        )
    
                        sdr_i = sdr.mean() - sdr_baseline.mean()
    
                        # Saving on a csv file
                        row = {
                            "snt_id": snt_id[0],
                            "sdr": sdr.mean(),
                            "sdr_i": sdr_i,
                            "si-snr": -sisnr.item(),
                            "si-snr_i": -sisnr_i.item(),
                        }
                        writer.writerow(row)
    
                        # Metric Accumulation
                        all_sdrs.append(sdr.mean())
                        all_sdrs_i.append(sdr_i.mean())
                        all_sisnrs.append(-sisnr.item())
                        all_sisnrs_i.append(-sisnr_i.item())
                    else:
                        skip_cnt += 1
                        print(f"Warning: skipping silent target, this has happened {skip_cnt} times")
            row = {
                "snt_id": "avg",
                "sdr": np.array(all_sdrs).mean(),
                "sdr_i": np.array(all_sdrs_i).mean(),
                "si-snr": np.array(all_sisnrs).mean(),
                "si-snr_i": np.array(all_sisnrs_i).mean(),
            }
            writer.writerow(row)

        logger.info("Mean SISNR is {}".format(np.array(all_sisnrs).mean()))
        logger.info("Mean SISNRi is {}".format(np.array(all_sisnrs_i).mean()))
        logger.info("Mean SDR is {}".format(np.array(all_sdrs).mean()))
        logger.info("Mean SDRi is {}".format(np.array(all_sdrs_i).mean()))
        if(self.hparams.result_file_path != ""):
            with open(self.hparams.result_file_path, "a", newline="", encoding="utf-8") as metrics_csv:
                writer = csv.DictWriter(metrics_csv, fieldnames=["model_name", "n_epochs", "learning_rate", "chunk_size", "sample_rate", "sdr", "sdr_i", "si-snr", "si-snr_i"])
                row = {
                        "model_name": self.hparams.experiment_name,
                        "learning_rate": self.hparams.lr,
                        "n_epochs": self.hparams.N_epochs,
                        "chunk_size":self.hparams.chunk_size,
                        "sample_rate":self.hparams.sample_rate,
                        "sdr": np.array(all_sdrs).mean(),
                        "sdr_i": np.array(all_sdrs_i).mean(),
                        "si-snr": np.array(all_sisnrs).mean(),
                        "si-snr_i": np.array(all_sisnrs_i).mean(),
                    }
                writer.writerow(row)

    def save_audio(self, snt_id, mixture, targets, predictions):
        "saves the test audio (mixture, targets, and estimated sources) on disk"

        # Create output folder
        save_path = os.path.join(self.hparams.save_folder, "audio_results")
        if not os.path.exists(save_path):
            os.mkdir(save_path)

        for ns in range(self.hparams.num_sources):
            # Estimated source
            signal = predictions[0, :, ns]
            signal = signal / signal.abs().max()
            save_file = os.path.join(
                save_path, "item{}_source{}hat.wav".format(snt_id, ns + 1)
            )
            torchaudio.save(
                save_file, signal.unsqueeze(0).cpu(), self.hparams.sample_rate
            )

            # Original source
            signal = targets[0, :, ns]
            signal = signal / signal.abs().max()
            save_file = os.path.join(
                save_path, "item{}_source{}.wav".format(snt_id, ns + 1)
            )
            torchaudio.save(
                save_file, signal.unsqueeze(0).cpu(), self.hparams.sample_rate
            )

        # Mixture
        signal = mixture[0][0, :]
        signal = signal / signal.abs().max()
        save_file = os.path.join(save_path, "item{}_mix.wav".format(snt_id))
        torchaudio.save(
            save_file, signal.unsqueeze(0).cpu(), self.hparams.sample_rate
        )





if __name__ == "__main__":
    # Load hyperparameters file with command-line overrides
    hparams_file, run_opts, overrides = sb.parse_arguments(sys.argv[1:])
    with open(hparams_file, encoding="utf-8") as fin:
        hparams = load_hyperpyyaml(fin, overrides)

    # Initialize ddp (useful only for multi-GPU DDP training)
    sb.utils.distributed.ddp_init_group(run_opts)

    # Logger info
    logger = get_logger(__name__)

    # Create experiment directory
    sb.create_experiment_directory(
        experiment_directory=hparams["output_folder"],
        hyperparams_to_save=hparams_file,
        overrides=overrides,
    )

    # Update precision to bf16 if the device is CPU and precision is fp16
    if run_opts.get("device") == "cpu" and hparams.get("precision") == "fp16":
        hparams["precision"] = "bf16"

   

    # Brain class initialization
    separator = Separation(
        modules=hparams["modules"],
        opt_class=hparams["optimizer"],
        hparams=hparams,
        run_opts=run_opts,
        checkpointer=hparams["checkpointer"],
    )
 
    # Training
        
    # Usage with SpeechBrain
    train_data = MusDBDataset(hparams["db_path"], subset="train", split="train", target_sr=hparams["sample_rate"], chunk_size=hparams["chunk_size"])
    valid_data = MusDBDataset(hparams["db_path"], subset="train", split="valid", target_sr=hparams["sample_rate"], chunk_size=hparams["chunk_size"])
    test_data = MusDBDataset(hparams["db_path"], subset="test", target_sr=hparams["sample_rate"], chunk_size=hparams["chunk_size"])
    

    # Create DataLoader
    train_loader = sb.dataio.dataloader.make_dataloader(
        train_data,
        batch_size=hparams["batch_size"],
        collate_fn=sb.dataio.batch.PaddedBatch  # Handles variable lengths
    )

    valid_loader = sb.dataio.dataloader.make_dataloader(
        valid_data,
        batch_size=hparams["batch_size"],
        collate_fn=sb.dataio.batch.PaddedBatch  # Handles variable lengths
    )

    test_loader = sb.dataio.dataloader.make_dataloader(
        test_data,
        batch_size=hparams["batch_size"],
        collate_fn=sb.dataio.batch.PaddedBatch  # Handles variable lengths
    )
    # print("STARTING FIT")
    separator.fit(
        separator.hparams.epoch_counter,
        train_loader,
        valid_loader,
        train_loader_kwargs=hparams["dataloader_opts"],
        valid_loader_kwargs=hparams["dataloader_opts"],
    )

    # # Eval
    separator.evaluate(test_loader, min_key="si-snr")
    separator.save_results(test_loader)

Overwriting sepformer-train.py


In [10]:
!python3 sepformer-train.py sepformer-hparams.yaml --data_folder=db_path --result_file_path={result_file_path}

speechbrain.utils.quirks - Applied quirks (see `speechbrain.utils.quirks`): [allow_tf32, disable_jit_profiling]
speechbrain.utils.quirks - Excluded quirks specified by the `SB_DISABLE_QUIRKS` environment (comma-separated list): []
speechbrain.core - Beginning experiment!
speechbrain.core - Experiment folder: /notebooks/results/sepformer/1234
speechbrain.core - Info: precision arg from hparam file is used
speechbrain.core - Info: noprogressbar arg from hparam file is used
speechbrain.core - Gradscaler enabled: `True`
speechbrain.core - Using training precision: `--precision=fp16`
speechbrain.core - Using evaluation precision: `--eval_precision=fp32`
speechbrain.core - Separation Model Statistics:
* Total Number of Trainable Parameters: 6.7M
* Total Number of Parameters: 6.7M
* Trainable Parameters represent 100.0000% of the total size.
speechbrain.utils.checkpoints - Loading a checkpoint from /notebooks/results/sepformer/1234/save/CKPT+2025-04-18+06-18-25+00
speechbrain.utils.checkpoint

## Demucs


In [11]:
%%file demucsModels.py

import torch
from torch import nn
from speechbrain.nnet.CNN import Conv1d, ConvTranspose1d
# from speechbrain.nnet.activations import GLU
from speechbrain.lobes.models.beats import GLU_Linear
from torch.nn import GLU
from speechbrain.nnet.RNN import LSTM
from speechbrain.nnet.linear import Linear

class EncoderBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv = Conv1d(
            out_channels=out_channels,
            in_channels=in_channels,
            kernel_size=8,
            stride=4,
            # default_padding=2,
            skip_transpose=True,
        )
        self.glu_conv = Conv1d(
            out_channels=2*out_channels,
            in_channels=out_channels,
            kernel_size=1,
            stride=1,
            skip_transpose=True,
        )
        self.relu = torch.nn.ReLU()
        self.glu = GLU(dim=1)

    def forward(self, x):
        # print(x.size())
        x = self.relu(self.conv(x))
        # print("#########checking encoder")
        # print(x.size())
        # print(self.glu_conv(x).size())
        # print("#########encoder ended")
        x = self.glu(self.glu_conv(x))
        # print(x.size())
        return x

class DecoderBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.glu_conv = Conv1d(
            out_channels=2*in_channels,
            in_channels=in_channels,
            kernel_size=1,
            stride=1,
            skip_transpose=True,
        )
        self.conv_tr = ConvTranspose1d(
            out_channels=out_channels,
            in_channels=in_channels,  # After GLU split
            kernel_size=8,
            stride=4,
            # padding=2,
            # output_padding=2,
            skip_transpose=True,
        )
        self.glu = GLU(dim=1)
        self.relu = torch.nn.ReLU()

    def forward(self, x, skip=None):

        if skip is not None:

            # T changed after conv1d in encoder, fix it here
            T_x = x.size(-1)
            T_skip = skip.size(-1)

            # # Case 1: Decoder output is longer
            # if T_skip > T_x:
            #     # Center-trim decoder output
            #     start = (T_skip - T_x) // 2
            #     skip = skip[..., start : start + T_x]

            # # Case 2: Skip is longer
            # elif T_skip < T_x:
            #     # Center-pad decoder output
            #     pad = T_x - T_skip
            #     skip = nn.functional.pad(skip, (pad // 2, pad - pad // 2))

            if T_skip > T_x:
                # Trim from the end of 'skip' to match 'x'
                skip = skip[..., :T_x]  # Keeps the first T_x samples

            # Case 2: Skip is shorter than target (x)
            elif T_skip < T_x:
                # Pad zeros at the end of 'skip' to match 'x'
                pad = T_x - T_skip
                skip = nn.functional.pad(skip, (0, pad))


            x = x + skip




        x = self.glu(self.glu_conv(x))

        x = self.relu(self.conv_tr(x))

        return x




class SourceSeparator(nn.Module):
    def __init__(self, in_channels, out_channels=2, num_sources=4):
        """
        Args:
            C_in: Input channels from last decoder (typically 8)
            C_out: Output channels per source (2 for stereo)
            num_sources: Number of sources to separate (e.g. 4 for vocals, drums, bass, other)
        """
        super().__init__()
        # Final linear layer (no activation)
        self.output_proj = Linear(
            input_size=in_channels,
            n_neurons=num_sources * out_channels,  # 4 sources * 2 channels = 8
            bias=True
        )
        self.num_sources = num_sources
        self.out_channels = out_channels

    def forward(self, x):
        """
        Input: [batch, C_in, time]
        Output: [batch, num_sources, out_channels, time]
        """
        # Permute to [batch, time, features]
        # print(x.size())
        x = x.permute(0, 2, 1)

        # Project to source waveforms
        x = self.output_proj(x)  # [batch, time, num_sources*out_channels]
        # print(x.size())

        # Reshape to separated sources
        # x = x.view(x.size(0), -1, self.num_sources, self.out_channels)
        x = x.permute(0, 2, 1)
        # print(x.size())
        # Return as [batch, sources, time]
        return x

Overwriting demucsModels.py


In [12]:
%%file demucs-hparams.yaml

# ################################
# Model: Demucs for source separation
# https://hal.science/hal-02379796/document
# Dataset : Musdb
# ################################
# Basic parameters
seed: 1234
__set_seed: !apply:speechbrain.utils.seed_everything [!ref <seed>]

# Data params (unchanged from DPRNN)
data_folder: !PLACEHOLDER
result_file_path: !PLACEHOLDER

experiment_name: demucs
output_folder: !ref /notebooks/results/<experiment_name>/<seed>
train_log: !ref <output_folder>/train_log.txt
save_folder: !ref <output_folder>/save
train_data: !ref <output_folder>/train.json
valid_data: !ref <output_folder>/valid.json
test_data: !ref <output_folder>/test.json
skip_prep: False
db_path: !ref /notebooks/data


# Experiment params
precision: fp32
num_sources: 2

instrumental_classification: False
noprogressbar: False
save_audio: True
sample_rate: 8000
n_audio_to_save: 10
chunk_size: 30

####################### Training Parameters ####################################

N_epochs: 30
batch_size: 1
lr: 0.0005
clip_grad_norm: 5
loss_upper_lim: 999999
limit_training_signal_len: False
training_signal_len: 32000000


# Data augmentation (unchanged)
use_wavedrop: False
use_rand_shift: False
min_shift: -8000
max_shift: 8000


# Frequency/time drop (unchanged)
drop_freq: !new:speechbrain.augment.time_domain.DropFreq
    drop_freq_low: 0
    drop_freq_high: 1
    drop_freq_count_low: 1
    drop_freq_count_high: 3
    drop_freq_width: 0.05

drop_chunk: !new:speechbrain.augment.time_domain.DropChunk
    drop_length_low: 1000
    drop_length_high: 2000
    drop_count_low: 1
    drop_count_high: 5

threshold_byloss: True
threshold: -30

################ Demucs Specific Parameters #############################
## for Demucs V3/4
# # Fourier Transform Parameters
# n_fft: 2048
# hop_length: 512


kernel_size: 16
# kernel_stride: 8

# Dataloader options (unchanged)
dataloader_opts:
    batch_size: !ref <batch_size>
    num_workers: 3

######################## Network Definition ####################################


Encoder1: !new:demucsModels.EncoderBlock
    in_channels: 1
    # kernel_size: !ref <kernel_size>
    out_channels: 64


Encoder2: !new:demucsModels.EncoderBlock
    in_channels: 64
    out_channels: 128


Encoder3: !new:demucsModels.EncoderBlock
    in_channels: 128
    out_channels: 256


Encoder4: !new:demucsModels.EncoderBlock
    in_channels: 256
    out_channels: 512


Encoder5: !new:demucsModels.EncoderBlock
    in_channels: 512
    out_channels: 1024


Encoder6: !new:demucsModels.EncoderBlock
    in_channels: 1024
    out_channels: 2048




Decoder6: !new:demucsModels.DecoderBlock
    in_channels: 2048
    out_channels: 1024


Decoder5: !new:demucsModels.DecoderBlock
    in_channels: 1024
    out_channels: 512


Decoder4: !new:demucsModels.DecoderBlock
    in_channels: 512
    out_channels: 256


Decoder3: !new:demucsModels.DecoderBlock
    in_channels: 256
    out_channels: 128


Decoder2: !new:demucsModels.DecoderBlock
    in_channels: 128
    out_channels: 64


Decoder1: !new:demucsModels.DecoderBlock
    in_channels: 64
    out_channels: 4


Linear: !new:speechbrain.nnet.linear.Linear
    input_size: 4096
    bias: False
    n_neurons: 2048

BiLSTM: !new:speechbrain.nnet.RNN.LSTM
    hidden_size: 2048
    input_size: 2048
    num_layers: 2
    bidirectional: True
    # batch_first: True

LinearSeparator: !new:demucsModels.SourceSeparator
    in_channels: 4
    out_channels: 1
    num_sources: !ref <num_sources>


######################## Remaining Config ######################################
optimizer: !name:torch.optim.Adam
    lr: !ref <lr>
    weight_decay: 0

# loss: !name:speechbrain.nnet.losses.mse_loss
loss: !name:speechbrain.nnet.losses.get_si_snr_with_pitwrapper
# loss: !name:speechbrain.nnet.losses.l1_loss


lr_scheduler: !new:speechbrain.nnet.schedulers.ReduceLROnPlateau
    factor: 0.5
    patience: 2
    dont_halve_until_epoch: 50

epoch_counter: !new:speechbrain.utils.epoch_loop.EpochCounter
    limit: !ref <N_epochs>

modules:
    encoder1: !ref <Encoder1>
    encoder2: !ref <Encoder2>
    encoder3: !ref <Encoder3>
    encoder4: !ref <Encoder4>
    encoder5: !ref <Encoder5>
    encoder6: !ref <Encoder6>
    lstm: !ref <BiLSTM>
    linear: !ref <Linear>
    decoder6: !ref <Decoder6>
    decoder5: !ref <Decoder5>
    decoder4: !ref <Decoder4>
    decoder3: !ref <Decoder3>
    decoder2: !ref <Decoder2>
    decoder1: !ref <Decoder1>
    linearSeparator: !ref <LinearSeparator>

checkpointer: !new:speechbrain.utils.checkpoints.Checkpointer
    checkpoints_dir: !ref <save_folder>
    recoverables:
        encoder1: !ref <Encoder1>
        encoder2: !ref <Encoder2>
        encoder3: !ref <Encoder3>
        encoder4: !ref <Encoder4>
        encoder5: !ref <Encoder5>
        encoder6: !ref <Encoder6>
        lstm: !ref <BiLSTM>
        linear: !ref <Linear>
        decoder6: !ref <Decoder6>
        decoder5: !ref <Decoder5>
        decoder4: !ref <Decoder4>
        decoder3: !ref <Decoder3>
        decoder2: !ref <Decoder2>
        decoder1: !ref <Decoder1>
        linearSeparator: !ref <LinearSeparator>
        counter: !ref <epoch_counter>


train_logger: !new:speechbrain.utils.train_logger.FileTrainLogger
    save_file: !ref <train_log>

Overwriting demucs-hparams.yaml


In [13]:
%%file demucs-train.py
#!/usr/bin/env/python3
"""Recipe for training a neural speech separation system on the wsjmix
dataset. The system employs an encoder, a decoder, and a masking network.

To run this recipe, do the following:
> python train.py hparams/sepformer.yaml
> python train.py hparams/dualpath_rnn.yaml
> python train.py hparams/convtasnet.yaml

The experiment file is flexible enough to support different neural
networks. By properly changing the parameter files, you can try
different architectures. The script supports both wsj2mix and
wsj3mix.


Authors
 * Cem Subakan 2020
 * Mirco Ravanelli 2020
 * Samuele Cornell 2020
 * Mirko Bronzi 2020
 * Jianyuan Zhong 2020
"""
## CHECKPOINT
import csv
import os
import sys

import numpy as np
import torch
import torch.nn.functional as F
import torchaudio
from hyperpyyaml import load_hyperpyyaml
from tqdm import tqdm



import speechbrain as sb
import speechbrain.nnet.schedulers as schedulers
from speechbrain.utils.distributed import run_on_main
from speechbrain.utils.logger import get_logger
from speechbrain.nnet.CNN import Conv1d, ConvTranspose1d
# from speechbrain.nnet.activations import GLU
from speechbrain.lobes.models.beats import GLU_Linear
from torch.nn import GLU
from speechbrain.nnet.RNN import LSTM
from speechbrain.nnet.linear import Linear
from demucsModels import EncoderBlock, DecoderBlock
from speechbrain.nnet.losses import get_si_snr_with_pitwrapper
from dataset import MusDBDataset

from torch.utils.data import Dataset
import musdb
np.float_ = np.float64





# Define training procedure
class DemucsSeparation(sb.Brain):
    # def on_fit_start(self):


    def compute_forward(self, mix, targets, stage, noise=None):
        """Forward computations from the mixture to the separated signals."""

        # Unpack lists and put tensors in the right device
        mix, mix_lens = mix
        mix, mix_lens = mix.to(self.device), mix_lens.to(self.device)

        # Convert targets to tensor
        # print([targets[i][0].size() for i in range(self.hparams.num_sources)])
        targets = torch.cat(
            [targets[i][0].unsqueeze(-1) for i in range(self.hparams.num_sources)],
            dim=-1,
        ).to(self.device)
        
        mix=mix.unsqueeze(1)
        targets=targets.permute(0,2,1)
       

        mix_enc_1 = self.modules.encoder1(mix)
        mix_enc_2 = self.modules.encoder2(mix_enc_1)
        mix_enc_3 = self.modules.encoder3(mix_enc_2)
        mix_enc_4 = self.modules.encoder4(mix_enc_3)
        mix_enc_5 = self.modules.encoder5(mix_enc_4)
        mix_enc_6 = self.modules.encoder6(mix_enc_5)

        lstm_in = mix_enc_6.permute(0,2,1)
        lstm_out, _ = self.modules.lstm(lstm_in) # outputs both -- outputs as well as hidden states -- we dont need hidden states
        lin_out = self.modules.linear(lstm_out)
        lin_out = lin_out.permute(0,2,1)

        mix_dec_6 = self.modules.decoder6(lin_out, skip=mix_enc_6)
        mix_dec_5 = self.modules.decoder5(mix_dec_6, skip=mix_enc_5)
        mix_dec_4 = self.modules.decoder4(mix_dec_5, skip=mix_enc_4)
        mix_dec_3 = self.modules.decoder3(mix_dec_4, skip=mix_enc_3)
        mix_dec_2 = self.modules.decoder2(mix_dec_3, skip=mix_enc_2)
        mix_dec_1 = self.modules.decoder1(mix_dec_2, skip=mix_enc_1)

        mix_out = self.modules.linearSeparator(mix_dec_1)
        est_source = mix_out



        # T changed after conv1d in encoder, fix it here
        T_origin = targets.size(2)
        T_est = est_source.size(2)

        if T_origin > T_est:
            est_source = F.pad(est_source, (0, 0, T_origin - T_est))
        else:
            est_source = est_source[:, : , :T_origin]

        return est_source, targets

    def compute_objectives(self, predictions, targets):
        """Computes the sinr loss"""
        # print("comp obj")
        targets = targets.permute(0,2,1)
        predictions = predictions.permute(0,2,1)
        return self.hparams.loss(targets, predictions) # for pitwrapper
## CHECKPOINT
    def fit_batch(self, batch):
        """Trains one batch"""

        # Unpacking batch list
        mixture = batch.mix_sig
        targets = [batch.voc_sig, batch.inst_sig]

        with self.training_ctx:
            predictions, targets = self.compute_forward(
                mixture, targets, sb.Stage.TRAIN
            )

            loss = self.compute_objectives(predictions, targets)

            # hard threshold the easy dataitems
            if self.hparams.threshold_byloss:
                th = self.hparams.threshold
                loss = loss[loss > th]
                if loss.nelement() > 0:
                    loss = loss.mean()
            else:
                loss = loss.mean()

        if loss.nelement() > 0 and loss < self.hparams.loss_upper_lim:
            self.scaler.scale(loss).backward()
            if self.hparams.clip_grad_norm >= 0:
                self.scaler.unscale_(self.optimizer)
                torch.nn.utils.clip_grad_norm_(
                    self.modules.parameters(),
                    self.hparams.clip_grad_norm,
                )
            self.scaler.step(self.optimizer)
            self.scaler.update()
        else:
            self.nonfinite_count += 1
            logger.info(
                "infinite loss or empty loss! it happened {} times so far - skipping this batch".format(
                    self.nonfinite_count
                )
            )
            loss.data = torch.tensor(0.0).to(self.device)
        self.optimizer.zero_grad()

        return loss.detach().cpu()

    def evaluate_batch(self, batch, stage):
        """Computations needed for validation/test batches"""
        snt_id = batch.track_id
        mixture = batch.mix_sig
        targets = [batch.voc_sig, batch.inst_sig]


        with torch.no_grad():
            predictions, targets = self.compute_forward(mixture, targets, stage)
            loss = self.compute_objectives(predictions, targets)

        # Manage audio file saving
        if stage == sb.Stage.TEST and self.hparams.save_audio:
            if hasattr(self.hparams, "n_audio_to_save"):
                if self.hparams.n_audio_to_save > 0:
                    self.save_audio(snt_id, mixture, targets, predictions)
                    self.hparams.n_audio_to_save += -1
            else:
                self.save_audio(snt_id, mixture, targets, predictions)

        return loss.mean().detach()

    def on_stage_end(self, stage, stage_loss, epoch):
        """Gets called at the end of a epoch."""
        # Compute/store important stats
        stage_stats = {"si-snr": stage_loss}
        if stage == sb.Stage.TRAIN:
            self.train_stats = stage_stats

        # Perform end-of-iteration things, like annealing, logging, etc.
        if stage == sb.Stage.VALID:
            # Learning rate annealing
            if isinstance(
                self.hparams.lr_scheduler, schedulers.ReduceLROnPlateau
            ):
                current_lr, next_lr = self.hparams.lr_scheduler(
                    [self.optimizer], epoch, stage_loss
                )
                schedulers.update_learning_rate(self.optimizer, next_lr)
            else:
                # if we do not use the reducelronplateau, we do not change the lr
                current_lr = self.hparams.optimizer.optim.param_groups[0]["lr"]

            self.hparams.train_logger.log_stats(
                stats_meta={"epoch": epoch, "lr": current_lr},
                train_stats=self.train_stats,
                valid_stats=stage_stats,
            )
            self.checkpointer.save_and_keep_only(
                meta={"si-snr": stage_stats["si-snr"]}, min_keys=["si-snr"]
            )
        elif stage == sb.Stage.TEST:
            self.hparams.train_logger.log_stats(
                stats_meta={"Epoch loaded": self.hparams.epoch_counter.current},
                test_stats=stage_stats,
            )


    def save_results(self, test_loader):
        """This script computes the SDR and SI-SNR metrics and saves
        them into a csv file"""

        # This package is required for SDR computation
        from mir_eval.separation import bss_eval_sources

        # Create folders where to store audio
        save_file = os.path.join(self.hparams.output_folder, "test_results.csv")

        # Variable init
        all_sdrs = []
        all_sdrs_i = []
        all_sisnrs = []
        all_sisnrs_i = []
        csv_columns = ["snt_id", "sdr", "sdr_i", "si-snr", "si-snr_i"]

        # test_loader = sb.dataio.dataloader.make_dataloader(
        #     test_data, **self.hparams.dataloader_opts
        # )

        with open(save_file, "w", newline="", encoding="utf-8") as results_csv:
            writer = csv.DictWriter(results_csv, fieldnames=csv_columns)
            writer.writeheader()

            # Loop over all test sentence
            with tqdm(test_loader, dynamic_ncols=True) as t:
                for i, batch in enumerate(t):
                    # Apply Separation
                    mixture, mix_len = batch.mix_sig
                    snt_id = batch.track_id
                    targets = [batch.voc_sig, batch.inst_sig]


                    with torch.no_grad():
                        predictions, targets = self.compute_forward(
                            batch.mix_sig, targets, sb.Stage.TEST
                        )

                    # Compute SI-SNR
                    predictions = predictions.permute(0,2,1)
                    targets = targets.permute(0,2,1)
                    print(predictions.size())
                    print(targets.size())
                    # sisnr = self.compute_objectives(predictions, targets)
                    sisnr = get_si_snr_with_pitwrapper(predictions,targets)

                    # Compute SI-SNR improvement
                    mixture_signal = torch.stack(
                        [mixture] * self.hparams.num_sources, dim=-1
                    )
                    print("---------------------")
                    # print(mixture.size())
                    print(mixture_signal.size())

                    mixture_signal = mixture_signal.to(targets.device)
                    # sisnr_baseline = self.compute_objectives(
                    #     mixture_signal, targets
                    # )
                    sisnr_baseline = get_si_snr_with_pitwrapper(mixture_signal, targets)
                    sisnr_i = sisnr - sisnr_baseline

                    # Compute SDR
                    sdr, _, _, _ = bss_eval_sources(
                        targets[0].mean(dim=1).t().cpu().numpy(),
                        predictions[0].mean(dim=1).t().detach().cpu().numpy(),
                    )

                    sdr_baseline, _, _, _ = bss_eval_sources(
                        targets[0].mean(dim=1).t().cpu().numpy(),
                        mixture_signal[0].mean(dim=1).t().detach().cpu().numpy(),
                    )

                    sdr_i = sdr.mean() - sdr_baseline.mean()

                    # Saving on a csv file
                    row = {
                        "snt_id": snt_id[0],
                        "sdr": sdr.mean(),
                        "sdr_i": sdr_i,
                        "si-snr": -sisnr.item(),
                        "si-snr_i": -sisnr_i.item(),
                    }
                    writer.writerow(row)

                    # Metric Accumulation
                    all_sdrs.append(sdr.mean())
                    all_sdrs_i.append(sdr_i.mean())
                    all_sisnrs.append(-sisnr.item())
                    all_sisnrs_i.append(-sisnr_i.item())

                row = {
                    "snt_id": "avg",
                    "sdr": np.array(all_sdrs).mean(),
                    "sdr_i": np.array(all_sdrs_i).mean(),
                    "si-snr": np.array(all_sisnrs).mean(),
                    "si-snr_i": np.array(all_sisnrs_i).mean(),
                }
                writer.writerow(row)

        logger.info("Mean SISNR is {}".format(np.array(all_sisnrs).mean()))
        logger.info("Mean SISNRi is {}".format(np.array(all_sisnrs_i).mean()))
        logger.info("Mean SDR is {}".format(np.array(all_sdrs).mean()))
        logger.info("Mean SDRi is {}".format(np.array(all_sdrs_i).mean()))
        if(self.hparams.result_file_path != ""):
            with open(self.hparams.result_file_path, "a", newline="", encoding="utf-8") as metrics_csv:
                writer = csv.DictWriter(metrics_csv, fieldnames=["model_name", "n_epochs", "learning_rate", "chunk_size", "sample_rate", "sdr", "sdr_i", "si-snr", "si-snr_i"])
                row = {
                        "model_name": self.hparams.experiment_name,
                        "learning_rate": self.hparams.lr,
                        "n_epochs": self.hparams.N_epochs,
                        "chunk_size":self.hparams.chunk_size,
                        "sample_rate":self.hparams.sample_rate,
                        "sdr": np.array(all_sdrs).mean(),
                        "sdr_i": np.array(all_sdrs_i).mean(),
                        "si-snr": np.array(all_sisnrs).mean(),
                        "si-snr_i": np.array(all_sisnrs_i).mean(),
                    }
                writer.writerow(row)

    def save_audio(self, snt_id, mixture, targets, predictions):
        "saves the test audio (mixture, targets, and estimated sources) on disk"

        # Create output folder

        save_path = os.path.join(self.hparams.save_folder, "audio_results")
        if not os.path.exists(save_path):
            os.mkdir(save_path)
        targets = targets.permute(0,2,1)
        predictions = predictions.permute(0,2,1)
        print(snt_id)
        for ns in range(self.hparams.num_sources):
            # Estimated source\
            
            print("------- in here --------------")
            print(predictions.size())
            signal = predictions[0, :, ns]
            print(signal.size())
            signal = signal / signal.abs().max()
            # signal = signal / signal.abs().max(dim=1, keepdim=True)[0]
            print(signal.size())
            save_file = os.path.join(
                save_path, "item{}_source{}hat.wav".format(snt_id, ns + 1)
            )
            torchaudio.save(
                save_file, signal.unsqueeze(0).cpu(), self.hparams.sample_rate, channels_first=True
            )

            # Original source
            signal = targets[0, :, ns]
            signal = signal / signal.abs().max()
            save_file = os.path.join(
                save_path, "item{}_source{}.wav".format(snt_id, ns + 1)
            )
            torchaudio.save(
                save_file, signal.unsqueeze(0).cpu(), self.hparams.sample_rate, channels_first=True
            )

        # Mixture
        signal = mixture[0][0, :]
        signal = signal / signal.abs().max()
        print("here now")
        print(signal.size())
        save_file = os.path.join(save_path, "item{}_mix.wav".format(snt_id))
        torchaudio.save(
            save_file, signal.unsqueeze(0).cpu(), self.hparams.sample_rate
        )



if __name__ == "__main__":
    # Load hyperparameters file with command-line overrides
    hparams_file, run_opts, overrides = sb.parse_arguments(sys.argv[1:])
    with open(hparams_file, encoding="utf-8") as fin:
        hparams = load_hyperpyyaml(fin, overrides)

    # Initialize ddp (useful only for multi-GPU DDP training)
    sb.utils.distributed.ddp_init_group(run_opts)

    # Logger info
    logger = get_logger(__name__)

    # Create experiment directory
    sb.create_experiment_directory(
        experiment_directory=hparams["output_folder"],
        hyperparams_to_save=hparams_file,
        overrides=overrides,
    )

    # Update precision to bf16 if the device is CPU and precision is fp16
    if run_opts.get("device") == "cpu" and hparams.get("precision") == "fp16":
        hparams["precision"] = "bf16"



        # Usage with SpeechBrain
    train_data = MusDBDataset(hparams["db_path"], subset="train", split="train", target_sr=hparams["sample_rate"], chunk_size=hparams["chunk_size"])
    valid_data = MusDBDataset(hparams["db_path"], subset="train", split="valid", target_sr=hparams["sample_rate"], chunk_size=hparams["chunk_size"])
    test_data = MusDBDataset(hparams["db_path"], subset="test", target_sr=hparams["sample_rate"], chunk_size=hparams["chunk_size"])


    # Create DataLoader
    train_loader = sb.dataio.dataloader.make_dataloader(
        train_data,
        batch_size=hparams["batch_size"],
        collate_fn=sb.dataio.batch.PaddedBatch  # Handles variable lengths
    )

    valid_loader = sb.dataio.dataloader.make_dataloader(
        valid_data,
        batch_size=hparams["batch_size"],
        collate_fn=sb.dataio.batch.PaddedBatch  # Handles variable lengths
    )

    test_loader = sb.dataio.dataloader.make_dataloader(
        test_data,
        batch_size=hparams["batch_size"],
        collate_fn=sb.dataio.batch.PaddedBatch  # Handles variable lengths
    )


    # Brain class initialization
    separator = DemucsSeparation(
        modules=hparams["modules"],
        opt_class=hparams["optimizer"],
        hparams=hparams,
        run_opts=run_opts,
        checkpointer=hparams["checkpointer"],
    )


    # Training
    separator.fit(
        separator.hparams.epoch_counter,
        train_loader,
        valid_loader,
        train_loader_kwargs=hparams["dataloader_opts"],
        valid_loader_kwargs=hparams["dataloader_opts"],
    )

    # Eval
    separator.evaluate(test_loader, min_key="si-snr")
    separator.save_results(test_loader)
    ## CHECKPOINT

Overwriting demucs-train.py


In [14]:
!python demucs-train.py demucs-hparams.yaml --data_folder=db_path --result_file_path={result_file_path}

speechbrain.utils.quirks - Applied quirks (see `speechbrain.utils.quirks`): [disable_jit_profiling, allow_tf32]
speechbrain.utils.quirks - Excluded quirks specified by the `SB_DISABLE_QUIRKS` environment (comma-separated list): []
speechbrain.core - Beginning experiment!
speechbrain.core - Experiment folder: /notebooks/results/demucs/1234
speechbrain.core - Info: precision arg from hparam file is used
speechbrain.core - Info: noprogressbar arg from hparam file is used
speechbrain.core - Gradscaler enabled: `False`
speechbrain.core - Using training precision: `--precision=fp32`
speechbrain.core - Using evaluation precision: `--eval_precision=fp32`
speechbrain.core - DemucsSeparation Model Statistics:
* Total Number of Trainable Parameters: 243.3M
* Total Number of Parameters: 243.3M
* Trainable Parameters represent 100.0000% of the total size.
speechbrain.utils.checkpoints - Loading a checkpoint from /notebooks/results/demucs/1234/save/CKPT+2025-04-18+14-38-53+00
speechbrain.utils.epoch

In [37]:
!zip -r results_30e_checkpoints.zip /notebooks/results/

  adding: notebooks/results/ (stored 0%)
  adding: notebooks/results/sepformer/ (stored 0%)
  adding: notebooks/results/sepformer/1234/ (stored 0%)
  adding: notebooks/results/sepformer/1234/log.txt (deflated 83%)
  adding: notebooks/results/sepformer/1234/test_results.csv (deflated 41%)
  adding: notebooks/results/sepformer/1234/sepformer-train.py (deflated 73%)
  adding: notebooks/results/sepformer/1234/train_log.txt (deflated 82%)
  adding: notebooks/results/sepformer/1234/env.log (deflated 52%)
  adding: notebooks/results/sepformer/1234/.ipynb_checkpoints/ (stored 0%)
  adding: notebooks/results/sepformer/1234/.ipynb_checkpoints/test_results-checkpoint.csv (deflated 41%)
  adding: notebooks/results/sepformer/1234/.ipynb_checkpoints/train_log-checkpoint.txt (deflated 82%)
  adding: notebooks/results/sepformer/1234/save/ (stored 0%)
  adding: notebooks/results/sepformer/1234/save/audio_results/ (stored 0%)
  adding: notebooks/results/sepformer/1234/save/audio_results/itemBuitraker - 